In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader
from torch.autograd import Variable

%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import time

%load_ext autoreload
%autoreload 2

from caltech256 import Caltech256

In [2]:
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize(224),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ]),
    'test': transforms.Compose([
        transforms.Resize(224),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
       transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ]),
    'visual': transforms.Compose([
        transforms.Resize(224),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
    ]),
}
'''
# Replace Resize() with below in pytorch 0.2
transforms.Scale(256),
'''

data_dir = 'data/256_ObjectCategories'
#data_dir = '/datasets/Caltech256/256_ObjectCategories'

In [3]:
caltech256_train = Caltech256(data_dir, data_transforms['train'], train=True)
caltech256_test = Caltech256(data_dir, data_transforms['test'], train=False)

In [4]:
vgg16 = models.vgg16_bn(pretrained=True)

for param in vgg16.parameters():
    param.requires_grad = False

vgg16.classifier._modules['6'] = nn.Linear(in_features=4096, out_features=256)

In [5]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(vgg16.classifier._modules['6'].parameters())
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=30)

vgg16 = nn.DataParallel(vgg16)
vgg16 = vgg16.cuda()
#vgg16.load_state_dict(torch.load('vgg16_tf_2_epochs'))

In [7]:
def train_model(model, dataset, criterion, optimizer, scheduler, num_epochs, batch_size):
    start_time = time.time()
    model.train(True)
    dataset_size = dataset.__len__()
    
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
    
    for epoch in range(num_epochs):
        scheduler.step()
        running_loss = 0.
        running_corrects = 0
        
        for data in dataloader:
            inputs, labels = data
            inputs, labels = Variable(inputs.cuda()), Variable(labels.cuda())
            
            optimizer.zero_grad()
            outputs = model(inputs)
            _, preds = torch.max(outputs.data, 1)
            
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            running_loss += loss.data[0] * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)
            
        epoch_loss = running_loss / dataset_size
        epoch_acc = running_corrects / dataset_size
        print('%d epoch loss: %f    accuracy: %f%%' % (epoch, epoch_loss, epoch_acc*100))
        
    model.train(False)
    time_elapsed = time.time() - start_time
    print('Training comple in %dm, %ds' % (time_elapsed//60, time_elapsed%60))
    return model

In [8]:
vgg16 = train_model(vgg16, caltech256_test, criterion, optimizer, scheduler, num_epochs=5, batch_size=16)

0 epoch loss: 3.507745    accuracy: 31.250000%
1 epoch loss: 0.613639    accuracy: 84.179688%
2 epoch loss: 0.304489    accuracy: 91.796875%
3 epoch loss: 0.184602    accuracy: 95.263672%
4 epoch loss: 0.112488    accuracy: 97.656250%
Training comple in 2m, 30s


In [9]:
testloader = DataLoader(caltech256_test, batch_size=16)
correct_cnt = 0
cnt = 0
for data in testloader:
    inputs, labels = data
    inputs, labels = Variable(inputs.cuda()), Variable(labels.cuda())
    outputs = vgg16(inputs)
    _, preds = torch.max(outputs, 1)
    correct_cnt += torch.sum(preds.data == labels.data)
    
acc = correct_cnt / caltech256_test.__len__()
print('Test Set Accuracy: %f%%' % (acc*100))

Test Set Accuracy: 99.609375%


In [10]:
class_correct = list(0. for i in range(257))
class_total = list(0. for i in range(257))

for data in testloader:
    images, labels = data
    outputs = vgg16(Variable(images.cuda()))
    _, predicted = torch.max(outputs.data, 1)
    c = (predicted == labels.cuda())
    for i in range(labels.size(0)):
        label = labels[i]
        class_correct[label] += c[i]
        class_total[label] += 1
        
class_cnt = 0
for i in range(257):
    if class_total[i] > 0:
        print('Accuracy of class %d : %2d %%' % (i, 100*class_correct[i]/class_total[i]))
        class_cnt += 1

Accuracy of class 0 : 100 %
Accuracy of class 1 : 100 %
Accuracy of class 2 : 100 %
Accuracy of class 3 : 100 %
Accuracy of class 4 : 100 %
Accuracy of class 5 : 100 %
Accuracy of class 6 : 100 %
Accuracy of class 7 : 100 %
Accuracy of class 8 : 100 %
Accuracy of class 9 : 100 %
Accuracy of class 10 : 100 %
Accuracy of class 11 : 100 %
Accuracy of class 12 : 100 %
Accuracy of class 13 : 100 %
Accuracy of class 14 : 100 %
Accuracy of class 15 : 100 %
Accuracy of class 16 : 100 %
Accuracy of class 17 : 100 %
Accuracy of class 18 : 100 %
Accuracy of class 19 : 100 %
Accuracy of class 20 : 100 %
Accuracy of class 21 : 100 %
Accuracy of class 22 : 100 %
Accuracy of class 23 : 100 %
Accuracy of class 24 : 100 %
Accuracy of class 25 : 100 %
Accuracy of class 26 : 100 %
Accuracy of class 27 : 100 %
Accuracy of class 28 : 100 %
Accuracy of class 29 : 100 %
Accuracy of class 30 : 100 %
Accuracy of class 31 : 100 %
Accuracy of class 32 : 100 %
Accuracy of class 33 : 100 %
Accuracy of class 34 : 1

In [11]:
torch.save(vgg16.state_dict(), 'vgg16_tf_5_epochs')

In [16]:
testiter = iter(testloader)
images, labels = testiter.next()
outputs = vgg16(Variable(images.cuda()))
_, pred = torch.max(outputs.data, 1)
print(pred, labels)


  94
  63
  57
 232
  86
 185
 171
  25
 240
 216
  46
  17
  65
 176
 243
  66
[torch.cuda.LongTensor of size 16 (GPU 0)]
 
  94
  63
  57
 232
  86
 185
 171
  25
 240
 216
  46
  17
  65
 176
 243
  66
[torch.LongTensor of size 16]

